In [1]:
import collections
import os
import random
import tarfile
import torch
from torch import nn
import torchtext.vocab as Vocab
import torch.utils.data as Data

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
DATA_ROOT = "D:/workingspace/Github/datasets"   

In [3]:
fname = os.path.join(DATA_ROOT, "aclImdb_v1.tar.gz")
if not os.path.exists(os.path.join(DATA_ROOT, "aclImdb")):
    print("从压缩包解压...")
    with tarfile.open(fname, 'r') as f:
        f.extractall(DATA_ROOT)

In [8]:
from tqdm import tqdm
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def read_imdb(folder='train', data_root=DATA_ROOT+"/aclImdb"): 
    data = []
    for label in ['pos', 'neg']:
        folder_name = os.path.join(data_root, folder, label)
        for file in tqdm(os.listdir(folder_name)):
            with open(os.path.join(folder_name, file), 'rb') as f:
                review = f.read().decode('utf-8').replace('\n', '').lower()
                data.append([review, 1 if label == 'pos' else 0])
    random.shuffle(data)
    return data

train_data, test_data = read_imdb('train'), read_imdb('test')

100%|█████████████████████████████████████████████████████████████████████████| 12500/12500 [00:01<00:00, 12171.09it/s]


In [66]:
# print(train_data[:10])

In [12]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def get_tokenized_imdb(data):
    """
    data: list of [string, label]
    """
    def tokenizer(text):
        return [tok.lower() for tok in text.split(' ')]
    return [tokenizer(review) for review, _ in data]

In [13]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def get_vocab_imdb(data):
    tokenized_data = get_tokenized_imdb(data)
    counter = collections.Counter([tk for st in tokenized_data for tk in st])
    return Vocab.Vocab(counter, min_freq=5)

vocab = get_vocab_imdb(train_data)
'# words in vocab:', len(vocab)

('# words in vocab:', 46152)

In [32]:
print(vocab)

In [14]:
# 本函数已保存在d2lzh_torch包中方便以后使用
def preprocess_imdb(data, vocab):
    max_l = 500  # 将每条评论通过截断或者补0，使得长度变成500

    def pad(x):
        return x[:max_l] if len(x) > max_l else x + [0] * (max_l - len(x))

    tokenized_data = get_tokenized_imdb(data)
    features = torch.tensor([pad([vocab.stoi[word] for word in words]) for words in tokenized_data])
    labels = torch.tensor([score for _, score in data])
    return features, labels

In [15]:
batch_size = 64
train_set = Data.TensorDataset(*preprocess_imdb(train_data, vocab))
test_set = Data.TensorDataset(*preprocess_imdb(test_data, vocab))
train_iter = Data.DataLoader(train_set, batch_size, shuffle=True)
test_iter = Data.DataLoader(test_set, batch_size)

单星号（*）：*agrs
将所以参数以元组(tuple)的形式导入：

此外，单星号的另一个用法是解压参数列表：

双星号（**）：**kwargs
将参数以字典的形式导入

In [74]:
def foo(a, b=10, *args, **kwargs):
    print(a)
    print(b)
    print(args)
    print(kwargs)

In [75]:
foo(1, 2, 3, 4, e=5, f=6, g=7)

1
2
(3, 4)
{'e': 5, 'f': 6, 'g': 7}


In [16]:
for X, y in train_iter:
    print('X', X.shape, 'y', y.shape)
    break
'#batches:', len(train_iter)

X torch.Size([64, 500]) y torch.Size([64])


('#batches:', 391)

In [17]:
class BiRNN(nn.Module):
    def __init__(self, vocab, embed_size, num_hiddens, num_layers):
        super(BiRNN, self).__init__()
        self.embedding = nn.Embedding(len(vocab), embed_size)
        # bidirectional设为True即得到双向循环神经网络
        self.encoder = nn.LSTM(input_size=embed_size, 
                                hidden_size=num_hiddens, 
                                num_layers=num_layers,
                                bidirectional=True)
        # 初始时间步和最终时间步的隐藏状态作为全连接层输入
        self.decoder = nn.Linear(4*num_hiddens, 2)

    def forward(self, inputs):
        # inputs的形状是(批量大小, 词数)，因为LSTM需要将序列长度(seq_len)作为第一维，所以将输入转置后
        # 再提取词特征，输出形状为(词数, 批量大小, 词向量维度)
        embeddings = self.embedding(inputs.permute(1, 0))
        # rnn.LSTM只传入输入embeddings，因此只返回最后一层的隐藏层在各时间步的隐藏状态。
        # outputs形状是(词数, 批量大小, 2 * 隐藏单元个数)
        outputs, _ = self.encoder(embeddings) # output, (h, c)
        # 连结初始时间步和最终时间步的隐藏状态作为全连接层输入。它的形状为
        # (批量大小, 4 * 隐藏单元个数)。
        encoding = torch.cat((outputs[0], outputs[-1]), -1)
        outs = self.decoder(encoding)
        return outs

In [45]:
embed_size, num_hiddens, num_layers = 100, 100, 2
net = BiRNN(vocab, embed_size, num_hiddens, num_layers)

In [22]:
glove_vocab = Vocab.GloVe(name='6B', dim=100, cache=os.path.join(DATA_ROOT, "glove"))

In [23]:
# 本函数已保存在d2lzh_torch包中方便以后使用
def load_pretrained_embedding(words, pretrained_vocab):
    """从预训练好的vocab中提取出words对应的词向量"""
    embed = torch.zeros(len(words), pretrained_vocab.vectors[0].shape[0]) # 初始化为0
    oov_count = 0 # out of vocabulary
    for i, word in enumerate(words):
        try:
            idx = pretrained_vocab.stoi[word]
            embed[i, :] = pretrained_vocab.vectors[idx]
        except KeyError:
            oov_count += 1
    if oov_count > 0:
        print("There are %d oov words." % oov_count)
    return embed

net.embedding.weight.data.copy_(
    load_pretrained_embedding(vocab.itos, glove_vocab))
net.embedding.weight.requires_grad = False # 直接加载预训练好的, 所以不需要更新它

There are 21202 oov words.


In [48]:
lr, num_epochs = 0.01, 5
# 要过滤掉不计算梯度的embedding参数
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)
loss = nn.CrossEntropyLoss()
d2l.train(train_iter, test_iter, net, loss, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 0.5670, train acc 0.685, test acc 0.830, time 77.5 sec
epoch 2, loss 0.1327, train acc 0.895, test acc 0.869, time 77.8 sec
epoch 3, loss 0.0497, train acc 0.945, test acc 0.846, time 79.0 sec
epoch 4, loss 0.0235, train acc 0.969, test acc 0.853, time 80.8 sec
epoch 5, loss 0.0113, train acc 0.981, test acc 0.858, time 80.8 sec


In [53]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def predict_sentiment(net, vocab, sentence):
    """sentence是词语的列表"""
    device = list(net.parameters())[0].device
    sentence = torch.tensor([vocab.stoi[word] for word in sentence], device=device)
    label = torch.argmax(net(sentence.view((1, -1))), dim=1)
    print(sentence)
    print(net(sentence.view((1, -1))))
    print(label)
    return 'positive' if label.item() == 1 else 'negative'

In [57]:
# for i in vocab:
#     print(i)
# print(len(vocab))

In [54]:
predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'great']) # positive

tensor([10, 20,  7, 38, 88], device='cuda:0')
tensor([[-5.0771,  4.8076]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([1], device='cuda:0')


'positive'

In [56]:
predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'bad']) # negative

tensor([10, 20,  7, 38, 97], device='cuda:0')
tensor([[ 1.7767, -1.6910]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([0], device='cuda:0')


'negative'

In [58]:
print(net)

BiRNN(
  (embedding): Embedding(46152, 100)
  (encoder): LSTM(100, 100, num_layers=2, bidirectional=True)
  (decoder): Linear(in_features=400, out_features=2, bias=True)
)


In [71]:
print(preprocess_imdb(train_data, vocab))

(tensor([[ 2596,  4127,     4,  ...,     0,     0,     0],
        [   10,    20,    14,  ...,     0,     0,     0],
        [   10,  2337,    20,  ...,   180,     3, 27992],
        ...,
        [    9,  1710,   140,  ...,  9112,     2,  5837],
        [    9,   200,    30,  ...,     0,     0,     0],
        [ 2188,    79,    25,  ...,     6,    73,    57]]), tensor([1, 1, 0,  ..., 0, 0, 0]))


In [72]:
print(*preprocess_imdb(train_data, vocab))

tensor([[ 2596,  4127,     4,  ...,     0,     0,     0],
        [   10,    20,    14,  ...,     0,     0,     0],
        [   10,  2337,    20,  ...,   180,     3, 27992],
        ...,
        [    9,  1710,   140,  ...,  9112,     2,  5837],
        [    9,   200,    30,  ...,     0,     0,     0],
        [ 2188,    79,    25,  ...,     6,    73,    57]]) tensor([1, 1, 0,  ..., 0, 0, 0])
